# Gaussian Process

In [10]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.kernel_ridge import KernelRidge as KR

import pandas as pd
import git

In [11]:
import warnings
warnings.simplefilter('ignore')

## Process the Datasetes

In [12]:
# return data ever since first min_cases cases
def select_region(df, fips, min_deaths=-1):
    d = df.loc[df['fips'] == fips]
    deaths = np.where(d['deaths'].values > min_deaths)[0]
    if len(deaths) == 0:
        return []
    start = deaths[0]
    d = d[start:]
    return d

In [50]:
import utils2 as utils
df = utils.get_processed_df()
print(df.head)

<bound method NDFrame.head of               date      county       state     fips  cases  deaths  \
0       2020-01-21   Snohomish  Washington  53061.0      1       0   
1       2020-01-22   Snohomish  Washington  53061.0      1       0   
2       2020-01-23   Snohomish  Washington  53061.0      1       0   
3       2020-01-24        Cook    Illinois  17031.0      1       0   
4       2020-01-24   Snohomish  Washington  53061.0      1       0   
...            ...         ...         ...      ...    ...     ...   
121038  2020-05-06    Sublette     Wyoming  56035.0      3       0   
121039  2020-05-06  Sweetwater     Wyoming  56037.0     19       0   
121040  2020-05-06       Teton     Wyoming  56039.0     98       1   
121041  2020-05-06       Uinta     Wyoming  56041.0      8       0   
121042  2020-05-06    Washakie     Wyoming  56043.0      7       0   

        date_processed  
0                  0.0  
1                  1.0  
2                  2.0  
3                  3.0  
4   

## Gaussian Process with Erf prior

In [14]:
import erf_model_small_changes as em
import pymc3 as pm
import arviz as az

Initialize the Erf prior as a mean function class.

In [15]:
import theano.tensor as tt

class Erf(pm.gp.mean.Mean):
    def __init__(self, fit_func, params):
        # Erf.__init__(self)
        self.fit_func = fit_func
        self.params = params

    def __call__(self, X):
        X = X.reshape(1, -1)[0]
        return em.run_model(self.fit_func, self.params, X)

In [60]:
def predict_county(county_fips):
    
    # If the number of deaths has always been 0, 
    # simply return zeroes for all percentiles
    d = select_region(df, county_fips)
    
    if d == []:
        return np.zeros((91, 9))
    
    if np.max(d['deaths'].values) == 0:
        return np.zeros((91, 9))

    with pm.Model() as gp_covid_model:
        # Lengthscale
        ρ = pm.HalfCauchy('ρ', 5)
        η = pm.HalfCauchy('η', 5)

        # Fit the Erf model
        fit_func, popt, pcov, X_train, y_train, X_pred = em.fit_erf(df, county_fips)
        
        # Make sure predictions start from where we started training
        # X_pred = X_pred[X_pred >= X_train[0]]

        # Set the Erf prior
        M = Erf(fit_func, popt)
        K = (η**2) * pm.gp.cov.ExpQuad(1, ρ)

        # Maybe I should change σ depending on pcov...
        σ = pm.HalfNormal('σ', 50)

        covid_deaths_gp = pm.gp.Marginal(mean_func=M, cov_func=K)
        covid_deaths_gp.marginal_likelihood('covid_deaths', X=X_train.reshape(-1, 1), 
                               y=y_train, noise=σ)

    if len(popt) == 2:
        with gp_covid_model:
            gp_trace = pm.sample(100, tune=100, cores=2, random_seed=10)
    else:
        with gp_covid_model:
            gp_trace = pm.sample(1000, tune=2000, cores=2, random_seed=42)

    with gp_covid_model:
        covid_pred_noise = covid_deaths_gp.conditional("covid_pred_noise", X_pred.reshape(-1,1), pred_noise=True)
        gp_covid_samples = pm.sample_posterior_predictive(gp_trace, vars=[covid_pred_noise], samples=500, random_seed=42)

    all_covid_samples = np.diff(gp_covid_samples['covid_pred_noise'])
    all_covid_samples = np.array(all_covid_samples)

    all_deciles = np.transpose(np.array([np.percentile(all_covid_samples, per, axis=0) for per in np.arange(10, 100, 10)]))
    all_deciles[all_deciles < 0] = 0
    
    return all_deciles

In [58]:
def predict_all_counties(out_file):
    out_dates = utils.all_output_dates()
    out_fips, all_row_starts = utils.all_output_fips('sample_submission.csv')
    num_dates, num_fips = len(out_dates), len(out_fips)
    out = np.zeros((num_dates * num_fips, 9))
    # Go through each county one by one, perform our fit, and record predictions
    for fi, fips in enumerate(out_fips):
        print('Processing FIPS', fips)
        preds = predict_county(float(fips))
        # Indices are disjointed because we're recording a single FIPS on many different dates
        out[np.arange(fi, out.shape[0], num_fips)] = preds
    # Add in the header line
    out_lines = [','.join(['id'] + ['%d' % x for x in np.arange(10, 91, 10)]) + '\n']
    # Add in all other lines one at a time
    for row_head, row in zip(all_row_starts, out):
        out_lines.append(','.join([row_head] + ['%.2f' % val for val in row]) + '\n')
    with open(out_file, 'w') as f:
        f.writelines(out_lines)
    f.close()

In [59]:
predict_all_counties('erf_model_predictions.csv')

Processing FIPS 10001


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [σ, η, ρ]
Sampling 2 chains, 0 divergences: 100%|██████████| 6000/6000 [00:47<00:00, 127.08draws/s]
The acceptance probability does not match the target. It is 0.8985177152689896, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.
100%|██████████| 500/500 [00:05<00:00, 88.05it/s] 


Processing FIPS 10003


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [σ, η, ρ]
Sampling 2 chains, 1 divergences: 100%|██████████| 6000/6000 [00:31<00:00, 188.09draws/s]
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
100%|██████████| 500/500 [00:05<00:00, 83.95it/s] 


Processing FIPS 10005


...

Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [σ, η, ρ]
Sampling 2 chains, 190 divergences: 100%|██████████| 400/400 [00:04<00:00, 95.28draws/s] 
There were 98 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.5973543597389411, but should be close to 0.8. Try to increase the number of tuning steps.
There were 90 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.5529003265431671, but should be close to 0.8. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samples is smaller than 10% for some parameters.
100%|██████████| 500/500 [00:03<00:00, 134.10it/s]


Processing FIPS 13121


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [σ, η, ρ]
100%|██████████| 500/500 [00:04<00:00, 116.06it/s]

Processing FIPS 13123
Processing FIPS 13125


TypeError: list indices must be integers or slices, not str